In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import folium 
from folium import plugins
plt.rcParams['figure.figsize'] = 10, 12
import warnings
warnings.filterwarnings('ignore')

In [2]:
MA_data = pd.read_csv('https://raw.githubusercontent.com/aboullaite/Covid19-MA/master/stats/MA-times_series.csv')
by_cities = pd.read_csv('https://raw.githubusercontent.com/aboullaite/Covid19-MA/master/stats/cities.csv')
by_regions = pd.read_csv('https://raw.githubusercontent.com/aboullaite/Covid19-MA/master/stats/regions.csv')
global_confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

In [3]:
MA_data.head()

,Dates / التواريخ,Cases / الحالات,Recovered / تعافى,Deaths / الوفيات
0,02/03/2020,1,0,0
1,05/03/2020,2,0,0
2,10/03/2020,2,0,1
3,11/03/2020,6,0,1
4,13/03/2020,8,1,1


In [4]:
by_cities.head()

,City / المدينة,Region / المنطقة,Active Cases / الحالات النشطة,Total Deaths / إجمالي الوفيات,Total Recovered / إجمالي المعافين
0,Aïn Harrouda,Casablanca-Settat,0,0,0
1,Ben Yakhlef,Casablanca-Settat,0,0,0
2,Bouskoura,Casablanca-Settat,0,0,0
3,Casablanca,Casablanca-Settat,130,1,1
4,Médiouna,Casablanca-Settat,0,0,0


In [5]:
by_regions.head()

,Region / الجهة,Total Cases / إجمالي الحالات,Active Cases / الحالات النشطة,Total Deaths / إجمالي الوفيات,Total Recovered / إجمالي المعافين
0,Tanger-Tétouan-Al Hoceima,115,114,1,0
1,Oriental,86,83,3,0
2,Fès-Meknès,167,147,8,12
3,Rabat-Salé-Kénitra,209,203,4,2
4,Béni Mellal-Khénifra,41,38,2,1


In [6]:
MA_data.columns = ['Dates','Confirmed','Recovered','Deaths']
by_cities.columns = ['City','Region','Confirmed','Deaths','Recovered']
by_regions.columns = ['Region','Total Cases','Confirmed','Deaths','Recovered']

In [7]:
Latitudes = [35.3174,33.3260,33.7560,34.0980,32.4843,33.1010,31.8310,31.3860,29.7453,28.3870,26.7270,22.8170]
Longitudes = [-5.4657,-2.4220,-4.8450,-6.3280,-6.3199,-7.8060,-8.3170,-5.5470,-8.1436,-10.0910,-11.6610,-14.3610]
by_regions['Latitude'] = Latitudes
by_regions['Longitude'] = Longitudes

In [8]:
# remove frensh accents from data
cols = by_regions.select_dtypes(include=[np.object]).columns
by_regions[cols] = by_regions[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
by_regions

,Region,Total Cases,Confirmed,Deaths,Recovered,Latitude,Longitude
0,Tanger-Tetouan-Al Hoceima,115,114,1,0,35.3174,-5.4657
1,Oriental,86,83,3,0,33.3260,-2.4220
2,Fes-Meknes,167,147,8,12,33.7560,-4.8450
3,Rabat-Sale-Kenitra,209,203,4,2,34.0980,-6.3280
4,Beni Mellal-Khenifra,41,38,2,1,32.4843,-6.3199
5,Casablanca-Settat,404,393,8,3,33.1010,-7.8060
6,Marrakech-Safi,258,248,8,2,31.8310,-8.3170
7,Draa-Tafilalet,67,66,0,1,31.3860,-5.5470
8,Souss-Massa,20,17,2,1,29.7453,-8.1436
9,Guelmim-Oued Noun,1,1,0,0,28.3870,-10.0910


In [9]:
total_confirmed = MA_data['Confirmed'].max()
total_recovered = MA_data['Recovered'].max()
total_deaths = MA_data['Deaths'].max()

## Total Confirmed cases in Morocco {{total_confirmed}}
## Total Recovered cases in Morocco {{total_recovered}}
## Total Death cases in Morocco {{total_deaths}}

## Regions with most active cases

In [10]:
by_regions.iloc[:,:-2].style.background_gradient(cmap='Reds')

,Region,Total Cases,Confirmed,Deaths,Recovered
0,Tanger-Tetouan-Al Hoceima,115,114,1,0
1,Oriental,86,83,3,0
2,Fes-Meknes,167,147,8,12
3,Rabat-Sale-Kenitra,209,203,4,2
4,Beni Mellal-Khenifra,41,38,2,1
5,Casablanca-Settat,404,393,8,3
6,Marrakech-Safi,258,248,8,2
7,Draa-Tafilalet,67,66,0,1
8,Souss-Massa,20,17,2,1
9,Guelmim-Oued Noun,1,1,0,0


## Total Active Cases in regions

In [11]:
total_active_cases_regions = pd.DataFrame({'Region':by_regions['Region'],'Active Cases':by_regions['Confirmed']-abs(by_regions['Recovered'] - by_regions['Deaths'])})
total_active_cases_regions.style.background_gradient(cmap='Reds')

,Region,Active Cases
0,Tanger-Tetouan-Al Hoceima,113
1,Oriental,80
2,Fes-Meknes,143
3,Rabat-Sale-Kenitra,201
4,Beni Mellal-Khenifra,37
5,Casablanca-Settat,388
6,Marrakech-Safi,242
7,Draa-Tafilalet,65
8,Souss-Massa,16
9,Guelmim-Oued Noun,1


## Geographic Visualization

In [12]:
regions_coord = by_regions[['Region','Latitude','Longitude']].copy()
regions_coord

,Region,Latitude,Longitude
0,Tanger-Tetouan-Al Hoceima,35.3174,-5.4657
1,Oriental,33.3260,-2.4220
2,Fes-Meknes,33.7560,-4.8450
3,Rabat-Sale-Kenitra,34.0980,-6.3280
4,Beni Mellal-Khenifra,32.4843,-6.3199
5,Casablanca-Settat,33.1010,-7.8060
6,Marrakech-Safi,31.8310,-8.3170
7,Draa-Tafilalet,31.3860,-5.5470
8,Souss-Massa,29.7453,-8.1436
9,Guelmim-Oued Noun,28.3870,-10.0910


In [25]:
map = folium.Map(location=[30, -7], zoom_start=5,tiles='Stamenterrain')

for lat, lon, value, name in zip(by_regions['Latitude'], by_regions['Longitude'], by_regions['Confirmed'], by_regions['Region']):
    folium.CircleMarker([lat, lon], radius=value*0.1, popup = ('<strong>State</strong>: ' + str(name).capitalize() + '<br>''<strong>Total Cases</strong>: ' + str(value) + '<br>'),color='red',fill_color='red',fill_opacity=0.3 ).add_to(map)
map

In [24]:
# from urllib.request import urlopen
# import json
# with urlopen('https://raw.githubusercontent.com/aboullaite/Covid19-MA/master/ma-convid19-state.geojson') as response:
#     counties = json.load(response)

 
# fig = px.choropleth(by_regions, geojson=counties, locations='Region', color='Confirmed',
#                            color_continuous_scale="oranges",
#                            featureidkey="properties.Name-FR",
#                            range_color=(0, by_regions['Confirmed'].max()),
                           
#                            labels={'Region':'Confirmed Cases'},
#                           )
# fig.update_geos(fitbounds="locations", visible=False)

# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()


In [15]:

#fig = px.scatter_mapbox(by_regions, lat="Latitude", lon="Longitude", hover_name="Region", hover_data=["Confirmed", "Recovered","Deaths"],
#                        color_discrete_sequence=["red"], zoom=5, height=500)
#fig.update_layout(mapbox_style="open-street-map")
#fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
#fig.show()

In [16]:
by_regions

,Region,Total Cases,Confirmed,Deaths,Recovered,Latitude,Longitude
0,Tanger-Tetouan-Al Hoceima,115,114,1,0,35.3174,-5.4657
1,Oriental,86,83,3,0,33.3260,-2.4220
2,Fes-Meknes,167,147,8,12,33.7560,-4.8450
3,Rabat-Sale-Kenitra,209,203,4,2,34.0980,-6.3280
4,Beni Mellal-Khenifra,41,38,2,1,32.4843,-6.3199
5,Casablanca-Settat,404,393,8,3,33.1010,-7.8060
6,Marrakech-Safi,258,248,8,2,31.8310,-8.3170
7,Draa-Tafilalet,67,66,0,1,31.3860,-5.5470
8,Souss-Massa,20,17,2,1,29.7453,-8.1436
9,Guelmim-Oued Noun,1,1,0,0,28.3870,-10.0910


In [17]:

fig = go.Figure()
fig.add_trace(go.Bar(
    y=by_regions['Region'],
    x=by_regions['Confirmed'],
    name='Confirmed',
    orientation='h',
    marker=dict(
        color='rgba(222, 25, 27, 0.6)',
        line=dict(color='rgba(222, 25, 27, 0.6)', width=3)
    )
))
fig.add_trace(go.Bar(
    y=by_regions['Region'],
    x=by_regions['Recovered'],
    name='Recovered',
    orientation='h',
    marker=dict(
        color='rgba(80, 180, 50, 0.6)',
        line=dict(color='rgba(80, 180, 50, 0.6)', width=3)
    )
))

fig.update_layout(barmode='stack',
                  title='Covid19 Cases in Morocco Regions',
                   xaxis_title='Cases',
                   yaxis_title='Regions')
fig.show()

In [18]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=MA_data['Dates'], y=MA_data['Confirmed'], name='Confirmed',
                         line=dict(color='royalblue', width=4)))
fig.add_trace(go.Scatter(x=MA_data['Dates'], y=MA_data['Recovered'], name = 'Recovered',
                         line=dict(color='limegreen', width=4)))
fig.add_trace(go.Scatter(x=MA_data['Dates'], y=MA_data['Deaths'], name='Deaths',
                         line=dict(color='firebrick', width=4,
                              dash='dash')
))


fig.update_layout(title='Covid19 Cases in Morocco',
                   xaxis_title='Days',
                   yaxis_title='Total Cases')



fig.show()

In [23]:
# Confirmed Cases
fig = px.bar(MA_data,x="Dates",y="Confirmed",color="Confirmed",orientation="v",
             height=600,title="Confirmed Cases in Morocco",
             color_discrete_sequence=px.colors.cyclical.IceFire)
fig.update_layout(plot_bgcolor='rgb(230,230,230)')
fig.show()

# Recovered Cases
fig = px.bar(MA_data,x="Dates",y="Recovered",color="Recovered",orientation="v",
             height=600,title="Recovered Cases in Morocco",
             color_discrete_sequence=px.colors.sequential.Plasma)
fig.update_layout(plot_bgcolor='rgb(230,230,230)')
fig.show()

# Deaths Cases
fig = px.bar(MA_data,x="Dates",y="Deaths",color="Deaths",orientation="v",
             height=600,title="Deaths Cases in Morocco",
             color_discrete_sequence=px.colors.cyclical.mrybm)
fig.update_layout(plot_bgcolor='rgb(230,230,230)')
fig.show()